In [2]:
# 🧾 QEPC Project Backup (Fast / Filtered)
#
# This backup:
#   - Detects the project root (folder that has both data/ and notebooks/)
#   - Creates backups/<qepc_backup_YYYYMMDD_HHMMSS>.zip
#   - INCLUDES: code, notebooks, configs, light data
#   - SKIPS:
#       * data/raw (huge CSVs you can regenerate)
#       * notebooks/02_utilities/data/raw
#       * backups/ (older backups)
#       * .git, __pycache__, .ipynb_checkpoints
#       * any single file larger than MAX_FILE_MB

from pathlib import Path
import datetime
import zipfile
import os

# ----------------------------
# 1. Detect project root
# ----------------------------
here = Path.cwd().resolve()
project_root = None

for p in [here] + list(here.parents):
    if (p / "data").exists() and (p / "notebooks").exists():
        project_root = p
        break

if project_root is None:
    raise RuntimeError(
        "Could not detect project root. "
        "Open this notebook from inside C:/Users/wdors/qepc_project or a subfolder."
    )

print(f"✅ Project root detected: {project_root}")

# ----------------------------
# 2. Backup config
# ----------------------------

backups_dir = project_root / "backups"
backups_dir.mkdir(exist_ok=True)
print(f"📁 Backups directory: {backups_dir}")

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
backup_name = f"qepc_backup_{timestamp}.zip"
backup_path = backups_dir / backup_name

# Maximum single-file size to include (in MB)
MAX_FILE_MB = 60
MAX_FILE_BYTES = MAX_FILE_MB * 1024 * 1024

print(f"🗂️ Backup file will be created as: {backup_path}")
print(f"⚙️ Max single-file size: {MAX_FILE_MB} MB\n")

# ----------------------------
# 3. Include / exclude rules
# ----------------------------

def should_exclude(path: Path) -> bool:
    """
    Return True if this path should be excluded from the backup.
    Decisions are based on the path *relative* to project_root.
    """
    rel = path.relative_to(project_root)
    parts = rel.parts

    # Skip backup folder itself
    if parts[0] == "backups":
        return True

    # Skip git metadata / caches / checkpoints
    if parts[0] in {".git", "__pycache__"}:
        return True
    if ".ipynb_checkpoints" in parts:
        return True

    # Skip virtualenvs if any live inside the project
    if parts[0] in {"venv", ".venv", "env", ".env"}:
        return True

    # Skip heavy raw data – these are regenerable
    #  - data/raw/...
    if len(parts) >= 2 and parts[0] == "data" and parts[1] == "raw":
        return True

    #  - notebooks/02_utilities/data/raw/...
    if (
        len(parts) >= 4
        and parts[0] == "notebooks"
        and parts[1] == "02_utilities"
        and parts[2] == "data"
        and parts[3] == "raw"
    ):
        return True

    # Skip existing zip/tar archives inside project (other than the one we're making)
    if rel.suffix in {".zip", ".tar", ".gz", ".bz2"} and parts[0] != "data":
        # You can adjust this rule if needed
        return True

    return False

# ----------------------------
# 4. Collect files to back up
# ----------------------------

files_to_add = []
skipped_large = []
skipped_rules = []

for path in project_root.rglob("*"):
    if not path.is_file():
        continue

    if should_exclude(path):
        skipped_rules.append(path)
        continue

    try:
        size = path.stat().st_size
    except OSError:
        continue

    if size > MAX_FILE_BYTES:
        skipped_large.append((path, size))
        continue

    files_to_add.append((path, size))

total_size_bytes = sum(size for _, size in files_to_add)
total_size_mb = total_size_bytes / (1024 * 1024)

print(f"📦 Files to include: {len(files_to_add)}")
print(f"📦 Approx compressed input size: {total_size_mb:.2f} MB")
print(f"🚫 Skipped by rules: {len(skipped_rules)} files")
print(f"🚫 Skipped as too large (> {MAX_FILE_MB} MB): {len(skipped_large)} files\n")

# ----------------------------
# 5. Create the ZIP
# ----------------------------

with zipfile.ZipFile(backup_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for idx, (path, size) in enumerate(files_to_add, start=1):
        rel = path.relative_to(project_root)
        zf.write(path, rel)
        if idx % 100 == 0:
            print(f"  → Added {idx} files so far...")

print("\n✅ Backup complete!")
final_size_mb = backup_path.stat().st_size / (1024 * 1024)
print(f"🧾 Backup file: {backup_path.name} ({final_size_mb:.2f} MB)")

# ----------------------------
# 6. Show what was skipped (summary)
# ----------------------------

if skipped_large:
    print("\n⚠️ The following large files were skipped (size > "
          f"{MAX_FILE_MB} MB):")
    for path, size in skipped_large:
        print(f"  - {path.relative_to(project_root)}  ({size / (1024*1024):.2f} MB)")

print("\nDone.")


✅ Project root detected: C:\Users\wdors\qepc_project
📁 Backups directory: C:\Users\wdors\qepc_project\backups
🗂️ Backup file will be created as: C:\Users\wdors\qepc_project\backups\qepc_backup_20251202_230610.zip
⚙️ Max single-file size: 60 MB

📦 Files to include: 194
📦 Approx compressed input size: 3.09 MB
🚫 Skipped by rules: 1736 files
🚫 Skipped as too large (> 60 MB): 0 files

  → Added 100 files so far...

✅ Backup complete!
🧾 Backup file: qepc_backup_20251202_230610.zip (0.77 MB)

Done.
